In [18]:
import pandas as pd
import numpy as np

In [19]:
def predict_crypto_prices():
    # Load the dataset
    data = pd.read_csv('crypto_price.csv')

    # Set the initial number of days to use for the moving average
    window_size = 10

    # Calculate the moving average
    data['MA'] = data['Price'].rolling(window_size).mean()

    # Extrapolate the next 31 days of prices based on the moving average and the average change
    predictions = []
    for i in range(31):
        # Get the most recent window_size days of data
        last_window = data.tail(window_size)

        # Calculate the average change in price over the last window_size days
        average_change = np.mean(np.diff(last_window['Price']))

        # Extrapolate the next day's price based on the last moving average and the average change
        if i == 0:
            prediction = data['Price'].iloc[-1]
        else:
            # Update the window size based on the number of predictions made so far
            window_size = min(i*2, len(data)-1)

            # Calculate the new moving average and make the prediction
            data['MA'] = data['Price'].rolling(window_size).mean()
            prediction = data['MA'].iloc[-1] + average_change

        # Append the prediction to the list of predictions
        predictions.append(prediction)

        # Update the dataset with the new prediction
        new_date = pd.date_range(data['Date'].iloc[-1], periods=2, freq='D')[1]
        new_data = pd.DataFrame(
            {'Date': new_date, 'Price': prediction}, index=[data.index[-1]+1])
        data = pd.concat([data, new_data], ignore_index=False)

    return predictions[:30]

In [20]:
predictions = predict_crypto_prices()

print(len(predictions))

30
